In [2]:
from selenium import webdriver
import time
import threading
from utils import btcToTrade, showCycle, percent
from configs import logging, API_KEY, API_SECRET, MIN_PERCENT_REQUIRED, BRL_AMOUNT_TRADE, UPDATE_TICK_RATE, LOGIN_TOKEN
from datetime import datetime as dt
from biscoint_api_python import Biscoint
import playsound
from classes.Robots import WebRobot, BiscointRobot

In [4]:
# Initial configs

# Selenium
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
driver = webdriver.Firefox(options=options)

driver.get("https://biscoint.io")
driver.execute_script(f"window.localStorage.setItem('Meteor.loginToken','{LOGIN_TOKEN}');")
driver.get("https://biscoint.io/dashboard/portfolio")

#Biscoint api 
bsc = Biscoint(API_KEY, API_SECRET)

# Web Robot
web_robot = WebRobot(bsc, driver)
biscoint_robot = BiscointRobot(bsc)

In [12]:
def updateTick(cycle_count):
    global ticker
    global amount_btc_to_trade
    if cycle_count % UPDATE_TICK_RATE == 0:
        try:
            ticker = bsc.get_ticker()
            amount_btc_to_trade =  btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])
        except Exception as e:
            logging.error(f"Error on updating tick {e}")

# if spread is high, sleep, else speed up checks
def waitForNextCycle(calculated_percent):
    if calculated_percent < -0.1:
        time.sleep(sleep_time_offers)
    else:
        time.sleep(2)


In [22]:
#Return initial balance from Biscoint
initial_balance = last_balance = bsc.get_balance()
# Calculate the rate limit of request to Biscoint API
endpoints_meta = bsc.get_meta()
rate_limit_offer = endpoints_meta['endpoints']['offer']['post']['rateLimit']
sleep_time_offers = ((rate_limit_offer["windowMs"] / rate_limit_offer["maxRequests"]) / 1000) * 1.5
# Convert the BRL amount of trading to BTC
ticker = bsc.get_ticker()

amount_btc_to_trade = btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])

percent_record = -1 # Will refresh every time when a new positive spread is achived

logging.info(f"Initial balance: {initial_balance}")
# Arbitrage Cycle
cycle_count = 1


Initial balance: {'BRL': '11.63', 'BTC': '0.00099334', 'ETH': '0.000000000000'}


In [14]:

while True:
    try:
        start_time = dt.now()
        # Get buy and sell offers of this cycle
        buy = web_robot.get_offer(op='buy',amount=str(amount_btc_to_trade),is_quote=False)
        sell = web_robot.get_offer(op='sell',amount=str(amount_btc_to_trade),is_quote=False)

        # Calculate if arbitrage is possible
        calculated_percent = percent(buy['efPrice'],sell['efPrice'])

        showCycle(cycle_count,calculated_percent)
        
        if calculated_percent > percent_record:
            logging.info(f"Percent Record Reached!! : {calculated_percent} at {dt.now()}")
            percent_record = calculated_percent
        # If Arbitrage is possible, confirm offers
        if calculated_percent >= MIN_PERCENT_REQUIRED:
            logging.info(f"Arbitrage oportunity: buy:{buy['efPrice']}   sell:{sell['efPrice']}")
            playsound('beep.wav')
            
            #Execute orders
            if float(last_balance['BRL']) < float(BRL_AMOUNT_TRADE):
                executed_sell = web_robot.confirm_offer(sell['offerId'])
                executed_buy = web_robot.confirm_offer(buy['offerId'])
            else:
                executed_buy = web_robot.confirm_offer(buy['offerId'])
                executed_sell = web_robot.confirm_offer(sell['offerId'])

            last_balance = bsc.get_balance()
            logging.info(f"New Balance is: {last_balance}")

            break
            

        end_time = dt.now()
        seconds_elapsed = (end_time - start_time).total_seconds()
        logging.debug(f"Cycle took {seconds_elapsed} seconds")
        cycle_count +=1

        updateTick(cycle_count)
        waitForNextCycle(calculated_percent)
    except Exception as e:
        logging.error(e)

Percent Record Reached!! : -0.69 at 2021-10-10 21:21:23.115451
Percent Record Reached!! : -0.657 at 2021-10-10 21:21:37.788381
Cycle 20 | Last percent : -0.755 |


Erro ao rodar get_offer Message: [object Object]



'NoneType' object is not subscriptable
Cycle 40 | Last percent : -0.723 |


Erro ao rodar get_offer Message: [object Object]



'NoneType' object is not subscriptable
Percent Record Reached!! : -0.592 at 2021-10-10 21:25:14.151476
Cycle 60 | Last percent : -0.691 |


...

Erro ao rodar get_offer Message: [object Object]



'NoneType' object is not subscriptable


Erro ao rodar get_offer Message: [object Object]



'NoneType' object is not subscriptable


KeyboardInterrupt: 